# Make word graphs from csv

From a csv file with at least three columns, two with words and one with a frequency. 

In [7]:
import sqlite3
import pandas as pd

# Functions 

In [3]:
def query(db, sql, param=()):
    """ Query a sqlitedatabase with sql and param"""
    with sqlite3.connect(db) as con:
        cur = con.cursor()
        cur.execute(sql, param)
    return cur.fetchall()

In [29]:
def create_database_from_df(db, df):
    """
        db is the name of an empty database, df is a dataframe with columns named first, second, freq
        A table named ngram is created with two secondary tables
    """
    print('creating database from dataframe')
    with sqlite3.connect(db) as con:
        df[['first','second','freq']].to_sql('ngram', con)
    print('indexing main table')
    query(db, "create index '_ft_' on ngram (first, second)")
    query(db, "create index '_tf_' on ngram (second, first)")
    query(db, "create table firstfreq (first varchar, freq int)")
    query(db, "create table secondfreq (second varchar, freq int)")
    query(db, "insert into firstfreq select first, sum(freq) from ngram group by first")
    query(db, "insert into secondfreq select second, sum(freq) from ngram group by second")
    print('indexing secondary tables')
    query(db, "create index _ff_ on firstfreq (first, freq)")
    query(db, "create index _ftf_ on secondfreq (second, freq)")

In [56]:
def make_word_graph(db):
    print('create graph')
    query(db, "create table word_graph (first varchar, second varchar, freq int, pmi float)")
    query(db, "insert into word_graph select  a.first, a.second, sum(a.freq) as f, pow(a.freq,2)*1.0/ (f.freq * t.freq) as pmi from ngram as a, firstfreq as f, secondfreq as t where a.first = f.first and a.second = t.second group by a.first, a.second")
    print('indexing graph')
    query(db, "create index _gftp_ on word_graph (first, pmi)")
    query(db, "create index _gtfp_ on word_graph (second, pmi)")

## File to store word graph - change name as appropriate

In [15]:
galakse = "galakse.db"

## CSV-file structure

Specify name of file, and a mapping from the columns to "first", "second" and "freq". The graph is built of from these elements. If the CSV file has five columns and the first element is in 2nd position and the second is in 4th, with frequency in 5th, just indicate the mapping with a schematic description of the columns so that first,second and freq are matched up with the appropriate columns, for example like this: 
```
['p0', 'first', 'p2', 'second', 'freq']
```
The names for unused columns are arbitrary. This will effectively select the words and the weights from the csv. The code will aggregate first, second and freq if the input CSV file has more than three columns

### specify CSV

In [36]:
csv_data = {
    'file':"../../ngram_2021_coordination/coord-bok.csv",
    'columns': ['lang', 'first', 'coord', 'second','freq']
}

# Read CSV and aggregate the frequencies

If csv has header set `header = 0`, and if there is a separate (initial) index column, set `index_col = 0`

In [37]:
data = pd.read_csv(csv_data['file'], header = None, index_col = None)

In [38]:
data.columns

Int64Index([0, 1, 2, 3, 4], dtype='int64')

In [39]:
data.columns = csv_data['columns']

In [41]:
if len(data.columns) > 3:
    data = data.groupby(['first', 'second']).sum('freq').reset_index()

# Create basic word pairs

Store the dataframe in an sqlite-database with frequencies together with derived frequencies - table is called `ngram`. Words with frequencies from the first and second columns are extracted and added up into separate tables dubbed `firstfreq` and `secondfreq`.

In [43]:
create_database_from_df(galakse, data)

creating database from dataframe
indexing main table
indexing secondary tables


# Make the galaxy

The galaxy table is computed using the ngram-table and named `galaxy`. Indexed and ready to use.

In [54]:
query(galakse, "drop table galaxy")

[]

In [55]:
make_galaxy(galakse)

create galaxies
indexing galaxies


# Querying word graphs